In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import pandas as pd
import os
#TODO: move to .env file
os.environ['NCBI_API_KEY'] = '9b3142c8c09a8527c8d9bc616f5ff2813d08'
from metapub import PubMedFetcher
from tqdm.notebook import tqdm

In [4]:
data_directory_openalex = '../../../../data/03_openalex'

files = os.listdir(data_directory_openalex)
subjects = [file.split('_openalex')[0] for file in files]

In [5]:
files

['adhd_openalex.csv',
 'animal_depression_openalex.csv',
 'atypical_antipsychotics_openalex.csv',
 'calcium_channel_blockers_openalex.csv',
 'oral_hypoglycemics_openalex.csv',
 'pancreatic_surgery_openalex.csv']

In [6]:
subjects

['adhd',
 'animal_depression',
 'atypical_antipsychotics',
 'calcium_channel_blockers',
 'oral_hypoglycemics',
 'pancreatic_surgery']

In [7]:
openalex_datasets = {
    subjects[count]: pd.read_csv(f'{data_directory_openalex}/{file}')
    for count, file in enumerate(files)
}

In [8]:
def fill_missing_from_pubmed(dataframe: pd.DataFrame, subject: str) -> pd.DataFrame:

    df = dataframe.copy()
    missing_rows = df[df['title'].isnull() | df['abstract'].isnull()]

    for index, row in tqdm(missing_rows.iterrows(), total=missing_rows.shape[0], desc=subject, leave=True):

        try:
            metadata = PubMedFetcher().article_by_pmid(row['literature_id'])
            df.loc[index, 'title'] = metadata.title
            df.loc[index, 'abstract'] = metadata.abstract
        except:
            pass

    return df

In [9]:
filled_datasets = {subject: fill_missing_from_pubmed( #ignore
    value, subject) for subject, value in openalex_datasets.items()} #ignore

adhd:   0%|          | 0/102 [00:00<?, ?it/s]

animal_depression:   0%|          | 0/357 [00:00<?, ?it/s]

atypical_antipsychotics:   0%|          | 0/122 [00:00<?, ?it/s]

calcium_channel_blockers:   0%|          | 0/182 [00:00<?, ?it/s]

oral_hypoglycemics:   0%|          | 0/84 [00:00<?, ?it/s]

pancreatic_surgery:   0%|          | 0/4159 [00:00<?, ?it/s]

## Export

In [15]:
data_directory_uniform = '../../../../data/04_missing'

[dataframe.to_csv(f'{data_directory_uniform}/{subject}_pubmed.csv', index=False)
 for subject, dataframe in filled_datasets.items()];